In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as auc

In [2]:
data_train_trd = pd.read_csv('./训练数据集1/训练数据集_trd.csv')
data_train_beh = pd.read_csv('./训练数据集1/训练数据集_beh.csv')
test_trd = pd.read_csv('./b/评分数据集_trd_b.csv')
test_beh = pd.read_csv('./b/评分数据集_beh_b.csv')

## 对beh表的所有pg_no进行挖掘

In [3]:
def beh_pg_train(code):
    output_dict = {}
    with open('./训练数据集1/训练数据集_beh.csv') as f:
        for line_num, line in enumerate(f):
            if line_num >0:
                user = line.split(',')[0]
                pay_way = line.split(',')[2]
                Trx1_1 = 0
                if user in output_dict:
                    Trx1_1 += output_dict[user]
                    if pay_way == code:
                        Trx1_1 += 1
                        output_dict.update({user:Trx1_1}) # update当前指标加上原指标
                        continue
                else:
                    if pay_way == code:
                        Trx1_1 = 1
                    output_dict.update({user:Trx1_1})
    output_list = [{'user':k, code:v} for k, v in output_dict.items()]
    Trx1_1_df = pd.DataFrame(output_list)
    return Trx1_1_df

In [7]:
ind = list(data_train_beh.page_no.value_counts().index)

In [9]:
ind

['CQA',
 'CQE',
 'AAO',
 'TRN',
 'XAI',
 'MSG',
 'FTR',
 'SZA',
 'BWA',
 'CTR',
 'XAG',
 'CQD',
 'SZD',
 'CQC',
 'EGA',
 'GBA',
 'CQB',
 'LC0',
 'BWE',
 'JF2',
 'JJK',
 'EGB',
 'MTA',
 'FLS',
 'LCT',
 'FDA',
 'SYK',
 'JJD',
 'ZY1']

In [10]:
train_pg_no = beh_pg_train('CQA')

In [12]:
for code in ind[1:]:
    cur = beh_pg_train(code)
    train_pg_no = pd.merge(train_pg_no, cur, how='left', left_on='user', right_on='user')

In [13]:
train_pg_no.head()

,user,CQA,CQE,AAO,TRN,XAI,MSG,FTR,SZA,BWA,...,JF2,JJK,EGB,MTA,FLS,LCT,FDA,SYK,JJD,ZY1
0,U64F0C9,4,7,6,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,UA262F2,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,U7EDD34,3,4,0,0,7,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
3,UFF6271,17,0,0,0,0,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,U29A12A,19,12,8,0,6,6,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [14]:
def beh_pg_test(code):
    output_dict = {}
    with open('./b/评分数据集_beh_b.csv') as f:
        for line_num, line in enumerate(f):
            if line_num >0:
                user = line.split(',')[0]
                pay_way = line.split(',')[1]
                Trx1_1 = 0
                if user in output_dict:
                    Trx1_1 += output_dict[user]
                    if pay_way == code:
                        Trx1_1 += 1
                        output_dict.update({user:Trx1_1}) # update当前指标加上原指标
                        continue
                else:
                    if pay_way == code:
                        Trx1_1 = 1
                    output_dict.update({user:Trx1_1})
    output_list = [{'user':k, code:v} for k, v in output_dict.items()]
    Trx1_1_df = pd.DataFrame(output_list)
    return Trx1_1_df

In [16]:
test_pg_no = beh_pg_test('CQA')

In [19]:
for code in ind[1:]:
    cur = beh_pg_test(code)
    test_pg_no = pd.merge(test_pg_no, cur, how='left', left_on='user', right_on='user')

In [20]:
# beh训练集
user_pgTimes = pd.DataFrame(data_train_beh.id.value_counts())
user_pgTimes['index'] = user_pgTimes.index
user_pgTimes.index = range(len(user_pgTimes))
user_pgTimes.columns = ['pg_times', 'user']
user_pgTimes = user_pgTimes[['user', 'pg_times']]

In [21]:
# beh测试集
test_user_pgTimes = pd.DataFrame(test_beh.id.value_counts())
test_user_pgTimes['index'] = test_user_pgTimes.index
test_user_pgTimes.index = range(len(test_user_pgTimes))
test_user_pgTimes.columns = ['pg_times', 'user']
test_user_pgTimes = test_user_pgTimes[['user', 'pg_times']]
test_user_pgTimes.shape

(1232, 2)

In [22]:
train_beh = pd.merge(user_pgTimes, train_pg_no, how='left', left_on='user', right_on='user')
train_beh.shape

(11913, 31)

In [23]:
test_beh = pd.merge(test_user_pgTimes, test_pg_no, how='left', left_on='user', right_on='user')
test_beh.shape

(1232, 31)

## 合并数据 填充众数

In [24]:
train_tag = pd.read_csv('./训练数据集1/train_tag_Ori.csv')
test_tag = pd.read_csv('./b/test_tag_Ori.csv')

In [25]:
train_trd_mf = pd.read_csv('./tide_up/train_trd_mf.csv')
test_trd_mf = pd.read_csv('./tide_up/test_trd_mf.csv')

In [27]:
train_data_mf = pd.merge(train_tag, train_trd_mf, how='left', left_on='id', right_on='user')
train_data_mf.drop(['user'], axis=1, inplace=True)
train_data_mf.shape

(39923, 55)

In [28]:
train_data_mf = pd.merge(train_data_mf, train_beh, how='left', left_on='id', right_on='user')
train_data_mf.drop(['user'], axis=1, inplace=True)
train_data_mf.shape

(39923, 85)

In [37]:
train_data_mf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39923 entries, 0 to 39922
Data columns (total 85 columns):
id                                 39923 non-null object
flag                               39923 non-null float64
gdr_cd                             39923 non-null float64
age                                39923 non-null int64
mrg_situ_cd                        39923 non-null float64
edu_deg_cd                         39923 non-null float64
acdm_deg_cd                        39923 non-null float64
deg_cd                             39923 non-null float64
job_year                           39923 non-null int64
ic_ind                             39923 non-null float64
fr_or_sh_ind                       39923 non-null float64
dnl_mbl_bnk_ind                    39923 non-null float64
dnl_bind_cmb_lif_ind               39923 non-null float64
hav_car_grp_ind                    39923 non-null float64
hav_hou_grp_ind                    39923 non-null float64
l6mon_agn_ind             

In [38]:
test_data_mf = pd.merge(test_tag, test_trd_mf, how='left', left_on='id', right_on='user')
test_data_mf.drop(['user'], axis=1, inplace=True)
test_data_mf.shape

(4000, 54)

In [39]:
test_data_mf = pd.merge(test_data_mf, test_beh, how='left', left_on='id', right_on='user')
test_data_mf.drop(['user'], axis=1, inplace=True)
test_data_mf.shape

(4000, 84)

In [40]:
train_data_ = train_data_mf.copy()
test_data_ = test_data_mf.copy()

for i in range(-20,0):
    train_data_.iloc[:, i].fillna(train_data_.iloc[:, i].mode()[0], inplace=True)
    test_data_.iloc[:, i].fillna(train_data_.iloc[:, i].mode()[0], inplace=True)

In [46]:
train_data_

,id,flag,gdr_cd,age,mrg_situ_cd,edu_deg_cd,acdm_deg_cd,deg_cd,job_year,ic_ind,...,JF2,JJK,EGB,MTA,FLS,LCT,FDA,SYK,JJD,ZY1
0,U7A4BAD,0.0,1.0,41,1.0,1.0,6.0,5.0,3,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,U557810,0.0,1.0,35,0.0,4.0,5.0,5.0,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,U1E9240,0.0,1.0,53,1.0,0.0,2.0,2.0,9,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,U6DED00,0.0,0.0,41,1.0,4.0,6.0,5.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,UDA8E28,0.0,0.0,42,1.0,1.0,6.0,1.0,3,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39918,U8E66E5,1.0,1.0,24,0.0,4.0,5.0,5.0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39919,U70BACF,0.0,1.0,41,1.0,4.0,5.0,5.0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39920,U777E7D,0.0,1.0,29,0.0,4.0,5.0,5.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39921,UE77AAD,0.0,1.0,31,0.0,4.0,1.0,5.0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [194]:
test_data_

,id,gdr_cd,age,mrg_situ_cd,edu_deg_cd,acdm_deg_cd,deg_cd,job_year,ic_ind,fr_or_sh_ind,...,JF2,JJK,EGB,MTA,FLS,LCT,FDA,SYK,JJD,ZY1
0,U1E63A3,1.0,40,1.0,2.0,1.0,5.0,1,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,U3EC77F,1.0,46,0.0,2.0,1.0,5.0,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,U27DC0C,1.0,42,1.0,4.0,5.0,5.0,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,U3FAAAC,0.0,35,1.0,1.0,0.0,5.0,5,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,U04EA0B,1.0,34,0.0,4.0,5.0,5.0,9,0.0,0.0,...,0.0,0.0,1.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,U0FBFE2,0.0,58,1.0,1.0,0.0,0.0,29,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3996,U506AD2,1.0,50,1.0,2.0,1.0,5.0,25,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3997,U287B33,1.0,39,0.0,10.0,6.0,5.0,0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3998,U15134F,1.0,34,0.0,1.0,0.0,5.0,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 建模分析

In [47]:
train_data_1 = train_data_.copy()
X, y = train_data_1.iloc[:,2:], train_data_1['flag']

In [48]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.3, random_state=0)

In [49]:
import xgboost as xgb

dtrain = xgb.DMatrix(Xtrain, Ytrain)

In [50]:
dtest = xgb.DMatrix(Xtest, Ytest)

In [51]:
dfull = xgb.DMatrix(X,y)

In [173]:
params = {'booster': 'gbtree',
         'objective': 'binary:logistic',
         'eval_metric': 'auc',
         'max_depth': 3,
         'lambda': 100,
         'gamma':0.1,
         'subsample': 0.75,
         'colsample_bytree': 0.5,
         'colsample_bylevel': 1,
         'colsample_bynode': 0.75,
         'min_child_weight': 30,
         'eta': 0.03,
         'seed': 0,
         'silent': 1,
         'n_fold': 5,
         'scale_pos_weight':1}
params1 = {'booster': 'gbtree',
         'objective': 'binary:logistic',
         'eval_metric': 'auc',
         'max_depth': 3,
         'lambda': 300,
         'gamma':0.1,
         'subsample': 1,
         'colsample_bytree': 0.75,
         'colsample_bylevel': 0.5,
         'colsample_bynode': 1,
         'min_child_weight': 5,
         'eta': 0.028,
         'seed': 0,
         'silent': 1,
         'n_fold': 5,
         'scale_pos_weight':1}
params2 = {'booster': 'gbtree',
         'objective': 'binary:logistic',
         'eval_metric': 'auc',
         'max_depth': 3,
         'lambda': 300,
         'gamma':0.15,
         'subsample': 0.75,
         'colsample_bytree': 0.5,
         'colsample_bylevel': 1,
         'colsample_bynode': 0.75,
         'min_child_weight': 30,
         'eta': 0.03,
         'seed': 0,
         'silent': 1,
         'n_fold': 5,
         'scale_pos_weight':1}
params3 = {'booster': 'gbtree',
         'objective': 'binary:logistic',
         'eval_metric': 'auc',
         'max_depth': 3,
         'lambda': 80,
         'gamma':0.2,
         'subsample': 0.75,
         'colsample_bytree': 0.75,
         'colsample_bylevel': 0.5,
         'colsample_bynode': 1,
         'min_child_weight': 20,
         'eta': 0.028,
         'seed': 0,
         'silent': 1,
         'n_fold': 5,
         'scale_pos_weight':1}
num_round=600

In [120]:
bst = xgb.train(params, dtrain, num_round)
preds = bst.predict(dtest)
print('train:{}'.format(auc(Ytrain, bst.predict(dtrain))))
auc(Ytest, preds)

train:0.7647171088025331


0.7470036326356891

In [187]:
bst = xgb.train(params1, dtrain, num_round)
preds = bst.predict(dtest)
print('train:{}'.format(auc(Ytrain, bst.predict(dtrain))))
auc(Ytest, preds)

train:0.7595616032366613


0.7461310748689707

In [186]:
bst = xgb.train(params1, dtrain, 590)
preds = bst.predict(dtest)
print('train:{}'.format(auc(Ytrain, bst.predict(dtrain))))
auc(Ytest, preds)

train:0.759291232294717


0.7460148070457302

In [174]:
bst = xgb.train(params3, dtrain, num_round)
preds = bst.predict(dtest)
print('train:{}'.format(auc(Ytrain, bst.predict(dtrain))))
auc(Ytest, preds)

train:0.7648282344366911


0.747033662999102

In [141]:
bst = xgb.train(params2, dtrain, num_round)
preds = bst.predict(dtest)
print('train:{}'.format(auc(Ytrain, bst.predict(dtrain))))
auc(Ytest, preds)

train:0.7587315204383892


0.7459506530845297

In [98]:
from time import time
import datetime

# (y==1).sum()/len(y)

In [188]:
params_ = {'booster': 'gbtree',
         'objective': 'binary:logistic',
         'eval_metric': 'auc',
         'max_depth': 3,
         'lambda': 300,
         'gamma':0.1,
         'subsample': 1,
         'colsample_bytree': 0.75,
         'colsample_bylevel': 0.5,
         'colsample_bynode': 1,
         'min_child_weight': 5,
         'eta': 0.028,
         'seed': 0,
         'silent': 1,
         'n_fold': 5,
         'scale_pos_weight':1}
num_round=600

In [189]:
bst = xgb.train(params_, dfull, num_round)

In [190]:
import joblib

joblib.dump(bst, './b/xgboost-fintech-joblib_4.dat')

['./b/xgboost-fintech-joblib_4.dat']

In [192]:
loaded_model_joblib = joblib.load('./b/xgboost-fintech-joblib_4.dat')
preds = loaded_model_joblib.predict(dtest)

auc(Ytest, preds)

0.7626988503242761

In [195]:
data_test_ = test_data_.copy()

In [196]:
test_data_feature = data_test_.drop(['id'], axis=1)
user_id = data_test_['id']
test_data_feature.shape

(4000, 83)

In [197]:
dtest_a = xgb.DMatrix(test_data_feature)

In [198]:
data_test_['preds'] = loaded_model_joblib.predict(dtest_a)

In [199]:
output_a = data_test_.loc[:,['id', 'preds']]

In [200]:
output_a.to_csv('./b/submit_b_4.txt', sep='\t',header=None, index=False, encoding='utf8')

In [201]:
from sklearn.linear_model import LogisticRegression as LR

In [202]:
LR_ = LR()

In [203]:
LR_ = LR_.fit(Xtrain, Ytrain)

/home/esterl/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [205]:
Xtrain.isnull().sum()

gdr_cd         0
age            0
mrg_situ_cd    0
edu_deg_cd     0
acdm_deg_cd    0
              ..
LCT            0
FDA            0
SYK            0
JJD            0
ZY1            0
Length: 83, dtype: int64